0. Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import re

1. Pegar todas as palavras chave dos textos

In [51]:
STRING_PATH = "data/strings-de-busca-ods/"

In [52]:
def listar_arquivos(diretorio):
    if not os.path.isdir(diretorio):
        print(f"O diretório '{diretorio}' não existe.")
        return []
    return os.listdir(diretorio)

# Carregar arquivos
def carregar_arquivo_json(path, nome_arquivo):
    with open(path + nome_arquivo, "r", encoding='utf-8') as arquivo:
        return json.load(arquivo)
    

def write_json_file(data, file_path):
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)



In [53]:
# pegar_palavras_chave(carregar_arquvo_txt("ods02.txt"))

class Ods:
    def __init__(self, path, ods_file):
        self.ods_strings = self.carregar_arquivo_txt(path, ods_file)
        self.ods_file = ods_file
        self.keywords = self.pegar_palavras_chave()

    def pegar_palavras_chave(self):
        palavras_chave = re.findall(r'\{(.*?)\}', self.ods_strings)
        return palavras_chave
    
    def dict_ods(self):
        return {"ods": self.ods_file, "keywords": self.keywords}

    def carregar_arquivo_txt(self, path, file_name):
        with open(path + file_name, "r", encoding='utf-8') as arquivo:
            return arquivo.read()

In [54]:

result = []
for file in listar_arquivos(STRING_PATH):
    result.append(Ods(STRING_PATH, file).dict_ods())



In [55]:
write_json_file(result, "data/strings-de-busca-ods/ods_keywords.json")

In [56]:
pd.read_json("data/strings-de-busca-ods/ods_keywords.json")

,ods,keywords
0,ods01.txt,"[extrema pobreza, alívio da pobreza, erradicaç..."
1,ods02.txt,"[direitos de posse da terra, produtor de alime..."
2,ods03.txt,"[síndrome da criança espancada, doença cardiov..."
3,ods04.txt,"[escola, educação, educacional, frequência esc..."
4,ods05.txt,"[desigualdade de gênero, igualdade de gênero, ..."
5,ods06.txt,"[seguro, acesso à água, água potável, limpa, á..."
6,ods07.txt,"[eficiência energética, consumo de energia, tr..."
7,ods08.txt,"[crescimento econômico, política de desenvolvi..."
8,ods09.txt,"[crescimento industrial, diversificação indust..."
9,ods10.txt,"[política de reforma econômica, políticas de r..."


2. Agora eu preciso buscar as palavras-chave nos textos do dataframe

In [57]:
main_df = pd.read_csv("data/train_df.tsv", sep="\t")

In [58]:
main_df.head(1)

,data,uri,idioma,editor,programa,direito_acesso,departamento,area_cnpq,titulo,tipo,patrocinio,keywords,abstract,palavras_chave,resumo
0,2022-03-29,https://repositorio.pucsp.br/jspui/handle/hand...,por,Pontifícia Universidade Católica de São Paulo,Programa de Estudos Pós-Graduados em Tecnologi...,Acesso Aberto,Faculdade de Ciências Exatas e Tecnologia,CNPQ::ENGENHARIAS,Transformação digital na contabilidade: as nov...,Dissertação,Coordenação de Aperfeiçoamento de Pessoal de N...,Technological innovation | Accounting | Accoun...,This research discusses the impact of digital ...,Inovação tecnológica | Contabilidade | Contado...,Esta pesquisa discute o impacto da transformaç...


In [59]:
# Selecionando as colunas de onde posso achas as palavras chave
main_df[['departamento', 'area_cnpq','titulo','palavras_chave','resumo']].values[0]

array(['Faculdade de Ciências Exatas e Tecnologia', 'CNPQ::ENGENHARIAS',
       'Transformação digital na contabilidade: as novas competências do contador',
       'Inovação tecnológica | Contabilidade | Contador | Receita Federal | Economia | Direito Tributário',
       'Esta pesquisa discute o impacto da transformação digital nos departamentos de contabilidade das corporações. Inicia-se com as possíveis causas que aceleraram a transformação digital nas corporações brasileiras. Nesse trajeto avalia a importância do impulso que as alterações da Secretaria da Receita Federal do Brasil produziram nessa evolução. Pela propensão interdisciplinar avalia as interferências da economia, da tecnologia e do direito nessas transformações. A ciência contábil é uma ciência humana, assim sendo o estudo se orienta nos efeitos que essas alterações produzem no papel desempenhado pelo contabilista. E avalia as novas competências que o contabilista necessita para exercer suas novas tarefas. Por fim, esbo

In [60]:
def formatar_strings(lista_strings):
    # Juntar todas as strings da lista em uma única string
    texto_completo = ' '.join(lista_strings)
    # Remover caracteres não alfabéticos da língua portuguesa usando expressão regular
    texto_formatado = re.sub(r'[^a-zA-ZáàâãéèêíïóôõöúçñÁÀÂÃÉÈÊÍÏÓÔÕÖÚÇÑ]', ' ', texto_completo)
    texto_formatado = re.sub(r'\s{2,}', ' ', texto_formatado)
    return texto_formatado.lower()

In [61]:
formatar_strings(main_df[['departamento', 'area_cnpq','titulo','palavras_chave','resumo']].values[0])

'faculdade de ciências exatas e tecnologia cnpq engenharias transformação digital na contabilidade as novas competências do contador inovação tecnológica contabilidade contador receita federal economia direito tributário esta pesquisa discute o impacto da transformação digital nos departamentos de contabilidade das corporações inicia se com as possíveis causas que aceleraram a transformação digital nas corporações brasileiras nesse trajeto avalia a importância do impulso que as alterações da secretaria da receita federal do brasil produziram nessa evolução pela propensão interdisciplinar avalia as interferências da economia da tecnologia e do direito nessas transformações a ciência contábil é uma ciência humana assim sendo o estudo se orienta nos efeitos que essas alterações produzem no papel desempenhado pelo contabilista e avalia as novas competências que o contabilista necessita para exercer suas novas tarefas por fim esboça os efeitos disruptivos que esse contabilista produz com su

In [62]:
main_df['texto_formatado'] = main_df[['departamento', 'area_cnpq','titulo','palavras_chave','resumo']].apply(formatar_strings, axis=1)

In [63]:
main_df.texto_formatado[0]

'faculdade de ciências exatas e tecnologia cnpq engenharias transformação digital na contabilidade as novas competências do contador inovação tecnológica contabilidade contador receita federal economia direito tributário esta pesquisa discute o impacto da transformação digital nos departamentos de contabilidade das corporações inicia se com as possíveis causas que aceleraram a transformação digital nas corporações brasileiras nesse trajeto avalia a importância do impulso que as alterações da secretaria da receita federal do brasil produziram nessa evolução pela propensão interdisciplinar avalia as interferências da economia da tecnologia e do direito nessas transformações a ciência contábil é uma ciência humana assim sendo o estudo se orienta nos efeitos que essas alterações produzem no papel desempenhado pelo contabilista e avalia as novas competências que o contabilista necessita para exercer suas novas tarefas por fim esboça os efeitos disruptivos que esse contabilista produz com su

In [64]:
def find_matching_terms(ods, terms, text):
    present_keywords = []
    for term in terms:
        if term in text:
            present_keywords.append(term)
    return {"ods": ods, "present_keywords": present_keywords}

def find_all_matching_terms(text):
    return [find_matching_terms(i, result[i]['keywords'], text) for i in range(1, 17)]


In [65]:
main_df['matching_terms'] = main_df.texto_formatado.apply(find_all_matching_terms)

In [66]:
main_df.matching_terms[3]

[{'ods': 1, 'present_keywords': []},
 {'ods': 2, 'present_keywords': []},
 {'ods': 3, 'present_keywords': []},
 {'ods': 4, 'present_keywords': []},
 {'ods': 5, 'present_keywords': []},
 {'ods': 6, 'present_keywords': []},
 {'ods': 7, 'present_keywords': []},
 {'ods': 8, 'present_keywords': []},
 {'ods': 9, 'present_keywords': []},
 {'ods': 10, 'present_keywords': ['cidade']},
 {'ods': 11, 'present_keywords': []},
 {'ods': 12, 'present_keywords': []},
 {'ods': 13, 'present_keywords': []},
 {'ods': 14, 'present_keywords': []},
 {'ods': 15, 'present_keywords': ['sociedade']},
 {'ods': 16, 'present_keywords': []}]

In [67]:
main_df.to_csv('main_df.csv', index=False)